<a href="https://colab.research.google.com/github/ShruthiKannappan/Calamity_Magnitude_Predictor/blob/main/landslides.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

#Importing The Required Libraries

In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt


#Reading the data from the CSV File

In [332]:
data=pd.read_csv('Landslides.csv')
data.head()


,id,date,time,country,nearest_places,hazard_type,landslide_type,trigger,storm_name,fatalities,...,population,countrycode,continentcode,key,version,tstamp,changeset_id,latitude,longitude,geolocation
0,1,01/02/2007,NaN,United Kingdom,"Whitehaven, Cumbria",landslide,Landslide,Rain,NaN,1.0,...,24803.0,GB,EU,sGaiIKdcW,1.0,Tue Apr 01 2014 00:00:00 GMT+0000 (UTC),1.0,54.549400,-3.587500,"(54.549399999999999, -3.5874999999999999)"
1,2,01/03/2007,NaN,Peru,"Alto Mesapata, in the Pasco province",landslide,Complex,Rain,NaN,NaN,...,0.0,PE,SA,S95OsumK4,1.0,Tue Apr 01 2014 00:00:00 GMT+0000 (UTC),1.0,-10.383800,-75.426700,"(-10.383800000000001, -75.426699999999997)"
2,3,01/05/2007,NaN,Brazil,"Nova Friburgo, Rio de Janeiro",landslide,Landslide,Rain,NaN,7.0,...,153361.0,BR,SA,D5vlNdgB7,1.0,Tue Apr 01 2014 00:00:00 GMT+0000 (UTC),1.0,-22.282346,-42.536362,"(-22.282346350000001, -42.536362050000001)"
3,4,01/05/2007,NaN,Brazil,"Sumidouro, Rio de Janeiro",landslide,Landslide,Rain,NaN,2.0,...,11555.0,BR,SA,h5J8IFdc4,1.0,Tue Apr 01 2014 00:00:00 GMT+0000 (UTC),1.0,-22.045200,-42.673400,"(-22.045200000000001, -42.673400000000001)"
4,5,01/05/2007,NaN,Brazil,"Jundiai, Sao Paulo",landslide,Landslide,Rain,NaN,1.0,...,54415.0,BR,SA,s5Zb5FdB4,1.0,Tue Apr 01 2014 00:00:00 GMT+0000 (UTC),1.0,-23.231100,-46.613600,"(-23.231100000000001, -46.613599999999998)"


In [ ]:
data.columns


Index(['id', 'date', 'time', 'country', 'nearest_places', 'hazard_type',
       'landslide_type', 'trigger', 'storm_name', 'fatalities', 'injuries',
       'source_name', 'source_link', 'location_description',
       'location_accuracy', 'landslide_size', 'photos_link', 'cat_src',
       'cat_id', 'countryname', 'near', 'distance', 'adminname1', 'adminname2',
       'population', 'countrycode', 'continentcode', 'key', 'version',
       'tstamp', 'changeset_id', 'latitude', 'longitude', 'geolocation'],
      dtype='object')

#CLEANING DATA
#1)Dropping the columns which don't have useful information/relevant information for prediction
The data which is specific to the calamity event like id is not going to generate patterns that the model can learn.


Dropping these columns helps the model to focus on the important features for prediction

#2) Removing rows with NULL values/Not Available values.

In [ ]:
data.drop('storm_name',inplace=True,axis=1)
data.drop('source_name',inplace=True,axis=1)
data.drop('source_link',inplace=True,axis=1)
data.drop('photos_link',inplace=True,axis=1)
data.drop('cat_src',inplace=True,axis=1)
data.drop('adminname1',inplace=True,axis=1)
data.drop('adminname2',inplace=True,axis=1)
data.columns

Index(['id', 'date', 'time', 'country', 'nearest_places', 'hazard_type',
       'landslide_type', 'trigger', 'fatalities', 'injuries',
       'location_description', 'location_accuracy', 'landslide_size', 'cat_id',
       'countryname', 'near', 'distance', 'population', 'countrycode',
       'continentcode', 'key', 'version', 'tstamp', 'changeset_id', 'latitude',
       'longitude', 'geolocation'],
      dtype='object')

In [ ]:
data.drop('key',inplace=True,axis=1)
data.drop('country',inplace=True,axis=1)
data.drop('countryname',inplace=True,axis=1)
data.drop('hazard_type',inplace=True,axis=1)
data.drop('geolocation',inplace=True,axis=1)
data.head()



,id,date,time,nearest_places,landslide_type,trigger,fatalities,injuries,location_description,location_accuracy,...,near,distance,population,countrycode,continentcode,version,tstamp,changeset_id,latitude,longitude
0,1,01/02/2007,NaN,"Whitehaven, Cumbria",Landslide,Rain,1.0,NaN,NaN,Known_within_5_km,...,Whitehaven,0.22405,24803.0,GB,EU,1.0,Tue Apr 01 2014 00:00:00 GMT+0000 (UTC),1.0,54.549400,-3.587500
1,2,01/03/2007,NaN,"Alto Mesapata, in the Pasco province",Complex,Rain,NaN,NaN,NaN,Unknown,...,Huancabamba,12.26171,0.0,PE,SA,1.0,Tue Apr 01 2014 00:00:00 GMT+0000 (UTC),1.0,-10.383800,-75.426700
2,3,01/05/2007,NaN,"Nova Friburgo, Rio de Janeiro",Landslide,Rain,7.0,NaN,NaN,Known_within_25_km,...,Nova Friburgo,0.63016,153361.0,BR,SA,1.0,Tue Apr 01 2014 00:00:00 GMT+0000 (UTC),1.0,-22.282346,-42.536362
3,4,01/05/2007,NaN,"Sumidouro, Rio de Janeiro",Landslide,Rain,2.0,NaN,NaN,Known_within_10_km,...,Carmo,14.05144,11555.0,BR,SA,1.0,Tue Apr 01 2014 00:00:00 GMT+0000 (UTC),1.0,-22.045200,-42.673400
4,5,01/05/2007,NaN,"Jundiai, Sao Paulo",Landslide,Rain,1.0,NaN,NaN,Known_within_50_km,...,Mairipor√£,10.07582,54415.0,BR,SA,1.0,Tue Apr 01 2014 00:00:00 GMT+0000 (UTC),1.0,-23.231100,-46.613600


In [ ]:
data.drop('time',inplace=True,axis=1)
data.drop('id',inplace=True,axis=1)
# print(data.landslide_size.unique())

In [ ]:
data.drop('version',inplace=True,axis=1)
data.drop('changeset_id',inplace=True,axis=1)
data.drop('cat_id',inplace=True,axis=1)
data.drop('nearest_places',inplace=True,axis=1)
data.drop('near',inplace=True,axis=1)


In [ ]:
data['Timestamp']=pd.to_datetime(data['tstamp']).astype('int64')/ 10**18
# data['Timestamp']=list(np.array(data['Timestamp'])).timestamp()

In [ ]:
print(data.head())

         date landslide_type trigger  fatalities  injuries  \
0  01/02/2007      Landslide    Rain         1.0       NaN   
1  01/03/2007        Complex    Rain         NaN       NaN   
2  01/05/2007      Landslide    Rain         7.0       NaN   
3  01/05/2007      Landslide    Rain         2.0       NaN   
4  01/05/2007      Landslide    Rain         1.0       NaN   

  location_description   location_accuracy landslide_size  distance  \
0                  NaN   Known_within_5_km         Medium   0.22405   
1                  NaN             Unknown         Medium  12.26171   
2                  NaN  Known_within_25_km          Large   0.63016   
3                  NaN  Known_within_10_km         Medium  14.05144   
4                  NaN  Known_within_50_km         Medium  10.07582   

   population countrycode continentcode  \
0     24803.0          GB            EU   
1         0.0          PE            SA   
2    153361.0          BR            SA   
3     11555.0          BR   

In [ ]:
data.drop('tstamp',inplace=True,axis=1)
data.drop('date',inplace=True,axis=1)

In [ ]:
print(data.shape)

(9564, 14)


In [ ]:
data.dropna(how='any',inplace=True,axis=0)

In [ ]:
print(data.shape)

(1671, 14)


In [ ]:
print(data.shape)

(1671, 14)


As we can see some strings denote the same meaning but are different by capitalisation, underscore etc. This will create additional categories for the exact same category.

Hence replacing these strings with minor differences like underscore,spelling,format etc with a uniform label to make dataset even more meaningful

In [ ]:
from math import nan
data['location_accuracy'] = data['location_accuracy'].replace(['Known_within_15km', 'Known_within_15_km'], 15)
data['location_accuracy'] = data['location_accuracy'].replace(['Known_within_5km', 'Known_within_5_km'], 5)
data['location_accuracy'] = data['location_accuracy'].replace(['Known_within_25_km', 'Known_within_25km'], 25)
data['location_accuracy'] = data['location_accuracy'].replace(['Known_within_50km','Known_within_50_km'], 50)
data['location_accuracy'] = data['location_accuracy'].replace(['Exact_Location', 'Exact_location'], 0)
data['location_accuracy'] = data['location_accuracy'].replace(['Unknown'],nan)
data['location_accuracy'] = data['location_accuracy'].replace(['Known_within_1_km', 'Known_within_1km'], 1)
data['location_accuracy'] = data['location_accuracy'].replace(['Known_within_30_km', 'Known_within_30km'], 30)
data['location_accuracy'] = data['location_accuracy'].replace(['Known_within_200_km', 'Known_within_200','Known_within_200km'], 200)
data['location_accuracy'] = data['location_accuracy'].replace(['Known_within_20_km','Known_within_20km'], 20)
data['location_accuracy'] = data['location_accuracy'].replace(['Known_within_35_km', ], 35)
data['location_accuracy'] = data['location_accuracy'].replace(['Known_within_2km'], 2)
data['location_accuracy'] = data['location_accuracy'].replace(['Known_within_3km'], 3)
data['location_accuracy'] = data['location_accuracy'].replace(['Known_within_40km'], 40)
data['location_accuracy'] = data['location_accuracy'].replace(['Known_within_1000km'], 1000)
data['location_accuracy'] = data['location_accuracy'].replace(['Known_within_75km'], 75)
data['location_accuracy'] = data['location_accuracy'].replace(['Known_within_100km','Known_within_100_km'], 100)
data['location_accuracy'] = data['location_accuracy'].replace(['Known_within_0.5km'], 0.5)
data['location_accuracy'] = data['location_accuracy'].replace(['Known_within_150km'], 75)
data['location_accuracy'] = data['location_accuracy'].replace(['Known_within_250km'], 100)
data['location_accuracy'] = data['location_accuracy'].replace(['Known_within_500km'], 500)
data['location_accuracy'] = data['location_accuracy'].replace(['Known_within_10_km','Known_within_10_km','Known_within_10km'], 10)
data['location_accuracy'] = data['location_accuracy'].replace(['Known_within_350km'], 350)
data['location_accuracy'] = data['location_accuracy'].replace(['Known_within_4km'], 4)
print(data.shape)

(1671, 14)


In [ ]:
data.dropna(how='any',inplace=True)
print(data.shape)

(1581, 14)


In [ ]:
data['landslide_type'] = data['landslide_type'].replace(['Landslide','landslide'],'Landslide')
data['landslide_type'] = data['landslide_type'].replace(['mudslide'],'Mudslide')

In [ ]:
data['trigger'] = data['trigger'].replace(['unknown'],'Unknown')
data['trigger'] = data['trigger'].replace(['No_Apparent_trigger'],'No_Apparent_Trigger')
data['trigger'] = data['trigger'].replace(['monsoon'],'Monsoon')
data['trigger'] = data['trigger'].replace(['Continuous_rain'],'Continuous_Rain')

In [ ]:
data['location_description'] = data['location_description'].replace(['Other-_Construction_Zone'],'Construction_zone')
data['location_description'] = data['location_description'].replace(['above road','above_road'],'Above_road')
data['location_description'] = data['location_description'].replace(['unknown'],'Unknown')
data['location_description'] = data['location_description'].replace(['Above_railraod'],'Above_Railroad')

In [ ]:
data['landslide_size'] = data['landslide_size'].replace(['medium'],'Medium')
data['landslide_size'] = data['landslide_size'].replace(['large'],'Large')
data['landslide_size'] = data['landslide_size'].replace(['small'],'Small')

Since the number of country codes is large, We can use OrdinalEncoder for encoding this string data. using one hot encoding would increase the number of features by a large amount.

In [ ]:
from sklearn.preprocessing import OrdinalEncoder
oe=OrdinalEncoder()

In [ ]:
data['countrycode']=np.array(oe.fit_transform(pd.DataFrame(data.countrycode)))

In [ ]:

data.head()

,landslide_type,trigger,fatalities,injuries,location_description,location_accuracy,landslide_size,distance,population,countrycode,continentcode,latitude,longitude,Timestamp
83,Landslide,Monsoon,128.0,150.0,Urban_area,15.0,Very_large,2.48492,3920222.0,6.0,AS,22.35550,91.820500,1.450300
484,Landslide,Downpour,4.0,10.0,Above_river,5.0,Medium,10.29236,7506.0,62.0,AS,6.53090,125.461600,1.452275
783,Complex,Dam_Embankment_Collapse,277.0,33.0,Mine_construction,25.0,Very_large,6.02877,0.0,15.0,AS,35.88530,111.583700,1.439936
1036,Landslide,Downpour,4.0,3.0,Unknown,5.0,Medium,10.34155,24561.0,53.0,AS,10.00028,98.548662,1.396310
1187,Mudslide,Downpour,3.0,5.0,Unknown,50.0,Medium,1.16036,200421.0,19.0,SA,-0.31520,-78.972400,1.452634


#Segregating columns as numerical/categorical

In [ ]:
from pandas.core.reshape.merge import string
from sklearn.compose import make_column_selector as selector

numerical_columns=['fatalities','injuries','location_accuracy','distance','population','longitude','latitude','countrycode']
categorical_columns=['landslide_type','trigger','location_description','continentcode']

In [ ]:
for column in data.columns:
  print(column)
  print(type(data[column].unique()[0]))
  print(len(data[column].unique()))
  print(data[column].unique())

landslide_type
<class 'str'>
12
['Landslide' 'Complex' 'Mudslide' 'Rockfall' 'Riverbank_Collapse' 'Other'
 'Debris_Flow' 'Translational_Slide' 'Unknown' 'Snow_Avalanche'
 'Earthflow' 'Creep']
trigger
<class 'str'>
15
['Monsoon' 'Downpour' 'Dam_Embankment_Collapse' 'Tropical_Cyclone' 'Rain'
 'Continuous_Rain' 'Unknown' 'Mining_digging' 'Snowfall_snowmelt' 'Other'
 'Flooding' 'Construction' 'Earthquake' 'No_Apparent_Trigger'
 'Freeze_thaw']
fatalities
<class 'numpy.float64'>
54
[1.28e+02 4.00e+00 2.77e+02 3.00e+00 0.00e+00 1.00e+00 5.00e+00 1.60e+01
 2.00e+00 4.10e+01 6.00e+00 1.80e+01 2.30e+01 8.00e+00 1.20e+01 7.00e+00
 1.40e+01 1.10e+01 2.10e+03 1.00e+01 9.00e+00 1.30e+01 7.00e+01 1.51e+02
 1.74e+02 7.20e+01 1.90e+01 2.20e+01 2.10e+01 3.70e+01 1.00e+02 2.40e+02
 5.50e+01 1.08e+02 5.00e+01 9.20e+01 5.30e+01 4.20e+01 2.80e+01 6.00e+01
 5.20e+01 2.50e+02 6.40e+01 2.70e+01 1.70e+01 2.00e+01 5.40e+01 6.90e+01
 3.40e+01 3.50e+01 4.70e+01 3.80e+01 3.10e+01 8.40e+01]
injuries
<class 'numpy.fl

#Data processing in ML Pipeline
Using One Hot Encoder for categorical data and StandardScalar for numerical data.


Grouping them as one preprocessor step/component.

In [ ]:
from sklearn.preprocessing import OneHotEncoder, StandardScaler

categorical_preprocessor = OneHotEncoder(handle_unknown="ignore")
numerical_preprocessor = StandardScaler()

In [ ]:
from sklearn.compose import ColumnTransformer

preprocessor = ColumnTransformer(
    [
        ("one-hot-encoder", categorical_preprocessor, categorical_columns),
        ("standard_scaler", numerical_preprocessor, numerical_columns),
    ]
)

splitting train and test data

In [ ]:
from sklearn.model_selection import train_test_split

data_train, data_test = train_test_split(
    data, random_state=42
)
data_train=pd.DataFrame(data_train)
data_test=pd.DataFrame(data_test)

In [ ]:
print(data_train.head())
print(data_test.head())

     landslide_type           trigger  fatalities  injuries  \
6282       Rockfall  Tropical_Cyclone         0.0       0.0   
6133      Landslide   Continuous_Rain         5.0       0.0   
6164      Landslide   Continuous_Rain         0.0       0.0   
9523      Landslide          Downpour         3.0       4.0   
6267      Landslide          Downpour         0.0       0.0   

     location_description  location_accuracy landslide_size  distance  \
6282           Above_road               25.0         Medium  12.64492   
6133              Unknown               50.0          Small   1.45285   
6164           Above_road                5.0          Small   0.61653   
9523           Urban_area                1.0         Medium   1.51145   
6267           Above_road               25.0          Small  11.43282   

      population  countrycode continentcode  latitude  longitude  Timestamp  
6282    798634.0         62.0            AS   10.4203   123.8418   1.414443  
6133         0.0         1

splitting X and Y->

 X input data fed containing features of calamity ,
 Y magnitude/category of calamity

In [ ]:
X_train=data_train.drop('landslide_size',axis=1)
Y_train=data_train.landslide_size
X_test=data_test.drop('landslide_size',axis=1)
Y_test=data_test.landslide_size

#Building pipelines using SKLearn

Logistic Regression Model used in pipeline

In [ ]:
from sklearn.linear_model import LogisticRegression
from sklearn.pipeline import make_pipeline

model_log = make_pipeline(preprocessor, LogisticRegression(max_iter=500))
model_log

Pipeline(steps=[('columntransformer',
                 ColumnTransformer(transformers=[('one-hot-encoder',
                                                  OneHotEncoder(handle_unknown='ignore'),
                                                  ['landslide_type', 'trigger',
                                                   'location_description',
                                                   'continentcode']),
                                                 ('standard_scaler',
                                                  StandardScaler(),
                                                  ['fatalities', 'injuries',
                                                   'location_accuracy',
                                                   'distance', 'population',
                                                   'longitude', 'latitude',
                                                   'countrycode'])])),
                ('logisticregression', LogisticRegression(max_iter=500))])

training the pipeline with training data

In [ ]:
model_log.fit(X_train,Y_train)

Pipeline(steps=[('columntransformer',
                 ColumnTransformer(transformers=[('one-hot-encoder',
                                                  OneHotEncoder(handle_unknown='ignore'),
                                                  ['landslide_type', 'trigger',
                                                   'location_description',
                                                   'continentcode']),
                                                 ('standard_scaler',
                                                  StandardScaler(),
                                                  ['fatalities', 'injuries',
                                                   'location_accuracy',
                                                   'distance', 'population',
                                                   'longitude', 'latitude',
                                                   'countrycode'])])),
                ('logisticregression', LogisticRegression(max_iter=500))])

Prediction

In [ ]:
pred_log=model_log.predict(data_test.drop('landslide_size',axis=1))


Accuracy of the model

In [ ]:
print(model_log.score(data_test.drop('landslide_size',axis=1), data_test.landslide_size))

0.6161616161616161


In [ ]:
print(pred_log)

['Medium' 'Medium' 'Medium' 'Small' 'Medium' 'Medium' 'Medium' 'Medium'
 'Medium' 'Medium' 'Medium' 'Medium' 'Medium' 'Medium' 'Medium' 'Medium'
 'Medium' 'Medium' 'Medium' 'Medium' 'Medium' 'Medium' 'Medium' 'Small'
 'Medium' 'Large' 'Medium' 'Medium' 'Medium' 'Medium' 'Medium' 'Medium'
 'Medium' 'Medium' 'Medium' 'Medium' 'Medium' 'Medium' 'Medium' 'Medium'
 'Medium' 'Medium' 'Medium' 'Medium' 'Medium' 'Medium' 'Medium' 'Medium'
 'Medium' 'Medium' 'Medium' 'Medium' 'Medium' 'Small' 'Medium' 'Medium'
 'Medium' 'Medium' 'Medium' 'Medium' 'Medium' 'Medium' 'Small' 'unknown'
 'Medium' 'Small' 'Medium' 'Medium' 'Small' 'Medium' 'Medium' 'Medium'
 'Medium' 'Small' 'Medium' 'Medium' 'Medium' 'Medium' 'Medium' 'Medium'
 'Medium' 'Medium' 'Medium' 'Medium' 'Medium' 'Medium' 'Medium' 'Medium'
 'Medium' 'Medium' 'Medium' 'Medium' 'Medium' 'Medium' 'Small' 'Medium'
 'Medium' 'Medium' 'Medium' 'Medium' 'Medium' 'Medium' 'Medium' 'Medium'
 'Medium' 'Medium' 'Medium' 'Medium' 'Medium' 'Medium' 'Med

Decision Tree Classifier used in pipeline

In [ ]:
from sklearn.tree import DecisionTreeClassifier
from sklearn.pipeline import make_pipeline
dtc = DecisionTreeClassifier()
modeldtc = make_pipeline(preprocessor, dtc)
modeldtc

Pipeline(steps=[('columntransformer',
                 ColumnTransformer(transformers=[('one-hot-encoder',
                                                  OneHotEncoder(handle_unknown='ignore'),
                                                  ['landslide_type', 'trigger',
                                                   'location_description',
                                                   'continentcode']),
                                                 ('standard_scaler',
                                                  StandardScaler(),
                                                  ['fatalities', 'injuries',
                                                   'location_accuracy',
                                                   'distance', 'population',
                                                   'longitude', 'latitude',
                                                   'countrycode'])])),
                ('decisiontreeclassifier', DecisionTreeClassifier())])

In [ ]:
modeldtc.fit(X_train,Y_train)


Pipeline(steps=[('columntransformer',
                 ColumnTransformer(transformers=[('one-hot-encoder',
                                                  OneHotEncoder(handle_unknown='ignore'),
                                                  ['landslide_type', 'trigger',
                                                   'location_description',
                                                   'continentcode']),
                                                 ('standard_scaler',
                                                  StandardScaler(),
                                                  ['fatalities', 'injuries',
                                                   'location_accuracy',
                                                   'distance', 'population',
                                                   'longitude', 'latitude',
                                                   'countrycode'])])),
                ('decisiontreeclassifier', DecisionTreeClassifier())])

Accuracy


In [ ]:
preddtc=modeldtc.predict(X_test)
print(modeldtc.score(X_test,Y_test))

0.5757575757575758


Support Vector Machine pipeline

In [ ]:
from sklearn.svm import SVC
svc = SVC()
model_svc=make_pipeline(preprocessor, svc)
model_svc


Pipeline(steps=[('columntransformer',
                 ColumnTransformer(transformers=[('one-hot-encoder',
                                                  OneHotEncoder(handle_unknown='ignore'),
                                                  ['landslide_type', 'trigger',
                                                   'location_description',
                                                   'continentcode']),
                                                 ('standard_scaler',
                                                  StandardScaler(),
                                                  ['fatalities', 'injuries',
                                                   'location_accuracy',
                                                   'distance', 'population',
                                                   'longitude', 'latitude',
                                                   'countrycode'])])),
                ('svc', SVC())])

accuracy

In [ ]:
model_svc.fit(X_train,Y_train)
predsvc=model_svc.predict(X_test)
print(model_svc.score(X_test,Y_test))

0.6313131313131313


K Nearest Neighbours Classifier Pipeline

In [ ]:
from sklearn.neighbors import KNeighborsClassifier
knn = KNeighborsClassifier()
model_knn=make_pipeline(preprocessor,knn)
model_knn


Pipeline(steps=[('columntransformer',
                 ColumnTransformer(transformers=[('one-hot-encoder',
                                                  OneHotEncoder(handle_unknown='ignore'),
                                                  ['landslide_type', 'trigger',
                                                   'location_description',
                                                   'continentcode']),
                                                 ('standard_scaler',
                                                  StandardScaler(),
                                                  ['fatalities', 'injuries',
                                                   'location_accuracy',
                                                   'distance', 'population',
                                                   'longitude', 'latitude',
                                                   'countrycode'])])),
                ('kneighborsclassifier', KNeighborsClassifier())])

accuracy

In [ ]:
model_knn.fit(X_train, Y_train)
predknn=model_knn.predict(X_test)
print(model_knn.score(X_test,Y_test))

0.6287878787878788
